In [1]:
import pandas as pd

In [2]:
def movies_genres_insert(movie_names,genres_names):
    queries = ""
    for movie_name in movie_names:
        for genres_name in genres_names:
            query = f"""INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = '{movie_name}' AND g.name = '{genres_name}';

"""
            queries = queries + query
    return queries

In [3]:
print(movies_genres_insert(['Тайна Коко'],['Мультфильмы','Фэнтези','Комедия','Приключения','Семейные','Музыкальные']))

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Мультфильмы';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Фэнтези';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Комедия';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Приключения';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Тайна Коко' AND g.name = 'Семейные';

INSERT INTO "public.Movies_genres" (movie_id, genres_id)
SELECT m.id, g.id
FROM "public.Movies" m
JOIN "public.Genres" g ON m.name = 'Та

In [23]:
def prime_nominations_insert_2(member_names,movie_names,prime_names,nominations,years,isWon):
    member_movie_names = list(zip(member_names, movie_names))
    
    member_movie_names_unique = []
    for names in member_movie_names: 
        if names not in member_movie_names_unique:
            member_movie_names_unique.append(names)
    member_movie_ids_map = {}
    for i in range(len(member_movie_names_unique)):
        member_movie_ids_map[member_movie_names_unique[i]] = i
            
    prime_names_unique = []
    for name in prime_names: 
        if name not in prime_names_unique:
            prime_names_unique.append(name)
    prime_ids_map = {}
    for i in range(len(prime_names_unique)):
        prime_ids_map[prime_names_unique[i]] = i
    
    queries_0 = """DO $$
"""
    for i in range(len(member_movie_names_unique)):
        query = f"""DECLARE id_nominee_{i} INT;
"""
        queries_0 = queries_0 + query
    for i in range(len(prime_names_unique)):
        query = f"""DECLARE id_prime_{i} INT;
"""
        queries_0 = queries_0 + query
        
    queries_1 = """
BEGIN"""
    for i in range(len(member_movie_names_unique)):
        query_2 = ""
        query_1 = f"""
	SELECT nom.id INTO id_nominee_{i}
	FROM "public.Nominees" nom
	JOIN "public.Movies" m ON nom.movie_id = m.id"""
        if len(member_movie_names_unique[i][0]) > 0:
            query_2 = f"""
	JOIN "public.Film_crew_members" cm ON nom.film_crew_members_id = cm.id"""
        query_3 = f"""
	WHERE m.name = '{member_movie_names_unique[i][1]}' AND """
        if len(member_movie_names_unique[i][0]) > 0:
            query_4 = f"""cm.name = '{member_movie_names_unique[i][0]}';
"""
        else: query_4 = f"""nom.film_crew_members_id IS NULL;
"""
        queries_1 = queries_1 + query_1 + query_2 + query_3 + query_4
    
    queries_2 = ""
    for i in range(len(prime_names_unique)):
        query_1 = f"""
	SELECT p.id INTO id_prime_{i}
	FROM "public.Primes" p
	WHERE p.name = '{prime_names_unique[i]}';
"""
        queries_2 = queries_2 + query_1
        
    queries_3 = f"""
	INSERT INTO "public.Prime_nominations" (nominee_id, prime_id, nomination, year, "isWon") VALUES"""
    for i in range(len(member_names)):
        query = f"""
		(id_nominee_{member_movie_ids_map[member_movie_names[i]]}, id_prime_{prime_ids_map[prime_names[i]]}, '{nominations[i]}', {int(years[i])}, {isWon[i]})"""
        if i == len(member_names)-1: query_end = """;"""
        else: query_end = ""","""
        queries_3 = queries_3 + query + query_end
        
    queries_4 = """
END $$;"""
        
    return queries_0 + queries_1 + queries_2 + queries_3 + queries_4

In [35]:
excel_file = 'sql_prime_nominations.xlsx'

df_prime_nominations = pd.read_excel(excel_file)
df_prime_nominations = df_prime_nominations.fillna('')

print(df_prime_nominations)

                Люди                               Фильмы  \
0          Том Хэнкс                         Зеленая миля   
1          Том Хэнкс                         Форрест Гамп   
2          Том Хэнкс                         Форрест Гамп   
3          Том Хэнкс                         Форрест Гамп   
4          Том Хэнкс                         Форрест Гамп   
5          Том Хэнкс                         Форрест Гамп   
6          Том Хэнкс                         Форрест Гамп   
7                                            Зеленая миля   
8                                            Зеленая миля   
9                                            Зеленая миля   
10                                           Форрест Гамп   
11                                           Форрест Гамп   
12                                           Форрест Гамп   
13                                           Форрест Гамп   
14       Тим Роббинс                    Побег из Шоушенка   
15     Морган Фриман    

In [36]:
member_names = df_prime_nominations['Люди']
movie_names = df_prime_nominations['Фильмы']
prime_names = df_prime_nominations['Премии']
nominations = df_prime_nominations['Номинации']
years = df_prime_nominations['Год']
isWon = df_prime_nominations['Статус']

print(prime_nominations_insert_2(member_names,movie_names,prime_names,nominations,years,isWon))

DO $$
DECLARE id_nominee_0 INT;
DECLARE id_nominee_1 INT;
DECLARE id_nominee_2 INT;
DECLARE id_nominee_3 INT;
DECLARE id_nominee_4 INT;
DECLARE id_nominee_5 INT;
DECLARE id_nominee_6 INT;
DECLARE id_nominee_7 INT;
DECLARE id_nominee_8 INT;
DECLARE id_nominee_9 INT;
DECLARE id_nominee_10 INT;
DECLARE id_nominee_11 INT;
DECLARE id_nominee_12 INT;
DECLARE id_nominee_13 INT;
DECLARE id_nominee_14 INT;
DECLARE id_nominee_15 INT;
DECLARE id_nominee_16 INT;
DECLARE id_nominee_17 INT;
DECLARE id_nominee_18 INT;
DECLARE id_nominee_19 INT;
DECLARE id_prime_0 INT;
DECLARE id_prime_1 INT;
DECLARE id_prime_2 INT;
DECLARE id_prime_3 INT;
DECLARE id_prime_4 INT;
DECLARE id_prime_5 INT;
DECLARE id_prime_6 INT;
DECLARE id_prime_7 INT;
DECLARE id_prime_8 INT;

BEGIN
	SELECT nom.id INTO id_nominee_0
	FROM "public.Nominees" nom
	JOIN "public.Movies" m ON nom.movie_id = m.id
	JOIN "public.Film_crew_members" cm ON nom.film_crew_members_id = cm.id
	WHERE m.name = 'Зеленая миля' AND cm.name = 'Том Хэнкс';

	S

In [25]:
def nominees_insert_2(member_names,movie_names):
    member_names_unique = []
    for name in member_names: 
        if name not in member_names_unique and len(name) > 0:
            member_names_unique.append(name)
    member_ids_map = {}
    for i in range(len(member_names_unique)):
        member_ids_map[member_names_unique[i]] = i
        
    movie_names_unique = []
    for name in movie_names: 
        if name not in movie_names_unique and len(name) > 0:
            movie_names_unique.append(name)
    movie_ids_map = {}
    for i in range(len(movie_names_unique)):
        movie_ids_map[movie_names_unique[i]] = i
    
    
    queries_0 = """DO $$
"""
    for i in range(len(member_names_unique)):
        query = f"""DECLARE id_member_{i} INT;
"""
        queries_0 = queries_0 + query
    for i in range(len(movie_names_unique)):
        query = f"""DECLARE id_movie_{i} INT;
"""
        queries_0 = queries_0 + query
        
    queries_1 = """
BEGIN"""
    for i in range(len(member_names_unique)):
        query_1 = f"""
	SELECT cm.id INTO id_member_{i}
	FROM "public.Film_crew_members" cm
	WHERE cm.name = '{member_names_unique[i]}';
"""
        queries_1 = queries_1 + query_1
    for i in range(len(movie_names_unique)):
        query_1 = f"""
	SELECT m.id INTO id_movie_{i}
	FROM "public.Movies" m
	WHERE m.name = '{movie_names_unique[i]}';
"""
        queries_1 = queries_1 + query_1

    queries_2 = f"""
	INSERT INTO "public.Nominees" (film_crew_members_id, movie_id) VALUES"""
    for i in range(len(member_names)):
        query = ""
        if (len(member_names[i]) > 0) & (len(movie_names[i]) > 0):
            query = f"""
		(id_member_{member_ids_map[member_names[i]]}, id_movie_{movie_ids_map[movie_names[i]]})"""
        elif (len(member_names[i]) > 0) & (len(movie_names[i]) == 0):
            query = f"""
		(id_member_{member_ids_map[member_names[i]]}, null)"""
        elif (len(member_names[i]) == 0) & (len(movie_names[i]) > 0):
            query = f"""
		(null, id_movie_{movie_ids_map[movie_names[i]]})"""
        if i == len(member_names)-1: query_end = """;"""
        else: query_end = ""","""
        queries_2 = queries_2 + query + query_end
        
    queries_3 = """
END $$;"""
        
    return queries_0 + queries_1 + queries_2 + queries_3

In [33]:
excel_file = 'sql_nominees.xlsx'

df_nominees = pd.read_excel(excel_file)
df_nominees = df_nominees.fillna('')

print(df_nominees)

                Люди                               Фильмы
0          Том Хэнкс                         Зеленая миля
1                                            Зеленая миля
2          Том Хэнкс                         Форрест Гамп
3                                            Форрест Гамп
4        Тим Роббинс                    Побег из Шоушенка
5      Морган Фриман                    Побег из Шоушенка
6                                       Побег из Шоушенка
7      Франсуа Клюзе                                  1+1
8            Омар Си                                  1+1
9                                                     1+1
10        Лиам Нисон                      Список Шиндлера
11                                        Список Шиндлера
12  Мэттью МакКонахи                         Интерстеллар
13       Энн Хэтэуэй                         Интерстеллар
14                                           Интерстеллар
15       Элайджа Вуд  Властелин колец: Возвращение короля
16      Орланд

In [34]:
member_names = df_nominees['Люди']
movie_names = df_nominees['Фильмы']
print(nominees_insert_2(member_names,movie_names))

DO $$
DECLARE id_member_0 INT;
DECLARE id_member_1 INT;
DECLARE id_member_2 INT;
DECLARE id_member_3 INT;
DECLARE id_member_4 INT;
DECLARE id_member_5 INT;
DECLARE id_member_6 INT;
DECLARE id_member_7 INT;
DECLARE id_member_8 INT;
DECLARE id_member_9 INT;
DECLARE id_member_10 INT;
DECLARE id_movie_0 INT;
DECLARE id_movie_1 INT;
DECLARE id_movie_2 INT;
DECLARE id_movie_3 INT;
DECLARE id_movie_4 INT;
DECLARE id_movie_5 INT;
DECLARE id_movie_6 INT;
DECLARE id_movie_7 INT;

BEGIN
	SELECT cm.id INTO id_member_0
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Том Хэнкс';

	SELECT cm.id INTO id_member_1
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Тим Роббинс';

	SELECT cm.id INTO id_member_2
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Морган Фриман';

	SELECT cm.id INTO id_member_3
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Франсуа Клюзе';

	SELECT cm.id INTO id_member_4
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Омар Си';

	SELECT cm.id INTO id

In [28]:
def films_positions_insert_2(member_names,movie_names,positions_names):
    member_names_unique = []
    for name in member_names: 
        if name not in member_names_unique and len(name) > 0:
            member_names_unique.append(name)
    member_ids_map = {}
    for i in range(len(member_names_unique)):
        member_ids_map[member_names_unique[i]] = i
        
    movie_names_unique = []
    for name in movie_names: 
        if name not in movie_names_unique and len(name) > 0:
            movie_names_unique.append(name)
    movie_ids_map = {}
    for i in range(len(movie_names_unique)):
        movie_ids_map[movie_names_unique[i]] = i
    
    
    queries_0 = """DO $$
"""
    for i in range(len(member_names_unique)):
        query = f"""DECLARE id_member_{i} INT;
"""
        queries_0 = queries_0 + query
    for i in range(len(movie_names_unique)):
        query = f"""DECLARE id_movie_{i} INT;
"""
        queries_0 = queries_0 + query
        
    queries_1 = """
BEGIN"""
    for i in range(len(member_names_unique)):
        query_1 = f"""
	SELECT cm.id INTO id_member_{i}
	FROM "public.Film_crew_members" cm
	WHERE cm.name = '{member_names_unique[i]}';
"""
        queries_1 = queries_1 + query_1
    for i in range(len(movie_names_unique)):
        query_1 = f"""
	SELECT m.id INTO id_movie_{i}
	FROM "public.Movies" m
	WHERE m.name = '{movie_names_unique[i]}';
"""
        queries_1 = queries_1 + query_1

    queries_2 = f"""
	INSERT INTO "public.Films_positions" (member_id, movie_id, name) VALUES"""
    for i in range(len(member_names)):
        query = f"""
		(id_member_{member_ids_map[member_names[i]]}, id_movie_{movie_ids_map[movie_names[i]]}, '{positions_names[i]}')"""
        if i == len(member_names)-1: query_end = """;"""
        else: query_end = ""","""
        queries_2 = queries_2 + query + query_end
        
    queries_3 = """
END $$;"""
        
    return queries_0 + queries_1 + queries_2 + queries_3

In [29]:
excel_file = 'sql_movie_positions.xlsx'

df_movie_positions = pd.read_excel(excel_file)

print(df_movie_positions)

                   Люди                               Фильмы Должности
0             Том Хэнкс                         Зеленая миля     Актер
1             Том Хэнкс                         Форрест Гамп     Актер
2           Тим Роббинс                    Побег из Шоушенка     Актер
3         Морган Фриман                    Побег из Шоушенка     Актер
4         Франсуа Клюзе                                  1+1     Актер
5               Омар Си                                  1+1     Актер
6            Лиам Нисон                      Список Шиндлера     Актер
7      Мэттью МакКонахи                         Интерстеллар     Актер
8           Энн Хэтэуэй                         Интерстеллар     Актер
9           Элайджа Вуд  Властелин колец: Возвращение короля     Актер
10         Орландо Блум  Властелин колец: Возвращение короля     Актер
11        Эдвард Нортон                      Бойцовский клуб     Актер
12            Брэд Питт                      Бойцовский клуб     Актер
13  Хе

In [30]:
member_names = df_movie_positions['Люди']
movie_names = df_movie_positions['Фильмы']
positions_names = df_movie_positions['Должности']
print(films_positions_insert_2(member_names,movie_names,positions_names))

DO $$
DECLARE id_member_0 INT;
DECLARE id_member_1 INT;
DECLARE id_member_2 INT;
DECLARE id_member_3 INT;
DECLARE id_member_4 INT;
DECLARE id_member_5 INT;
DECLARE id_member_6 INT;
DECLARE id_member_7 INT;
DECLARE id_member_8 INT;
DECLARE id_member_9 INT;
DECLARE id_member_10 INT;
DECLARE id_member_11 INT;
DECLARE id_member_12 INT;
DECLARE id_movie_0 INT;
DECLARE id_movie_1 INT;
DECLARE id_movie_2 INT;
DECLARE id_movie_3 INT;
DECLARE id_movie_4 INT;
DECLARE id_movie_5 INT;
DECLARE id_movie_6 INT;
DECLARE id_movie_7 INT;

BEGIN
	SELECT cm.id INTO id_member_0
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Том Хэнкс';

	SELECT cm.id INTO id_member_1
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Тим Роббинс';

	SELECT cm.id INTO id_member_2
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Морган Фриман';

	SELECT cm.id INTO id_member_3
	FROM "public.Film_crew_members" cm
	WHERE cm.name = 'Франсуа Клюзе';

	SELECT cm.id INTO id_member_4
	FROM "public.Film_crew_members" c

In [48]:
def sessions_insert_(cinema_names,hall_names,movie_names,time_starts,time_ends):
    cinema_hall_names = list(zip(cinema_names, hall_names))
    
    cinema_hall_names_unique = []
    for names in cinema_hall_names: 
        if names not in cinema_hall_names_unique:
            cinema_hall_names_unique.append(names)
    cinema_hall_ids_map = {}
    for i in range(len(cinema_hall_names_unique)):
        cinema_hall_ids_map[cinema_hall_names_unique[i]] = i
            
    movie_names_unique = []
    for name in movie_names: 
        if name not in movie_names_unique:
            movie_names_unique.append(name)
    movie_ids_map = {}
    for i in range(len(movie_names_unique)):
        movie_ids_map[movie_names_unique[i]] = i
    
    queries_0 = """DO $$
"""
    for i in range(len(cinema_hall_names_unique)):
        query = f"""DECLARE id_hall_{i} INT;
"""
        queries_0 = queries_0 + query
    for i in range(len(movie_names_unique)):
        query = f"""DECLARE id_movie_{i} INT;
"""
        queries_0 = queries_0 + query
        
    queries_1 = """
BEGIN"""
    for i in range(len(cinema_hall_names_unique)):
        query_1 = f"""
	SELECT h.id INTO id_hall_{i}
	FROM "public.Halls" h
	JOIN "public.Cinemas" c ON h.cinema_id = c.id
	WHERE c.name = '{cinema_hall_names_unique[i][0]}' AND h.name = '{cinema_hall_names_unique[i][1]}';
"""
        queries_1 = queries_1 + query_1
    
    queries_2 = ""
    for i in range(len(movie_names_unique)):
        query_1 = f"""
	SELECT m.id INTO id_movie_{i}
	FROM "public.Movies" m
	WHERE m.name = '{movie_names_unique[i]}';
"""
        queries_2 = queries_2 + query_1
        
    queries_3 = f"""
	INSERT INTO "public.Sessions" (hall_id, movie_id, time_start, time_end) VALUES"""
    for i in range(len(cinema_names)):
        query = f"""
		(id_hall_{cinema_hall_ids_map[cinema_hall_names[i]]}, id_movie_{movie_ids_map[movie_names[i]]}, '{time_starts[i]}', '{time_ends[i]}')"""
        if i == len(cinema_names)-1: query_end = """;"""
        else: query_end = ""","""
        queries_3 = queries_3 + query + query_end
        
    queries_4 = """
END $$;"""
        
    return queries_0 + queries_1 + queries_2 + queries_3 + queries_4

In [49]:
excel_file = 'sql_sessions.xlsx'

df_sessions = pd.read_excel(excel_file)

print(df_sessions)

  Кинотеатр    Зал                                Фильм              Начало  \
0     Альфа  Г-401                         Зеленая миля 2023-11-03 11:00:00   
1     Альфа  Г-401                         Форрест Гамп 2023-11-03 14:30:00   
2     Альфа  Г-402                    Побег из Шоушенка 2023-11-03 12:00:00   
3     Альфа  Г-402                                  1+1 2023-11-03 15:00:00   
4     Альфа  Г-403                      Список Шиндлера 2023-11-03 13:00:00   
5     Альфа  Г-405                         Интерстеллар 2023-11-03 11:30:00   
6     Альфа  Г-407  Властелин колец: Возвращение короля 2023-11-03 12:30:00   
7     Альфа  Г-407                      Бойцовский клуб 2023-11-03 16:30:00   
8     Альфа  Г-408                 Унесённые призраками 2023-11-03 10:30:00   
9     Альфа  Г-408                           Тайна Коко 2023-11-03 13:30:00   

                Конец  
0 2023-11-03 14:10:00  
1 2023-11-03 17:00:00  
2 2023-11-03 14:30:00  
3 2023-11-03 17:00:00  
4 2023-11-

In [50]:
cinema_names = df_sessions['Кинотеатр']
hall_names = df_sessions['Зал']
movie_names = df_sessions['Фильм']
time_starts = df_sessions['Начало']
time_ends = df_sessions['Конец']
print(sessions_insert_(cinema_names,hall_names,movie_names,time_starts,time_ends))

DO $$
DECLARE id_hall_0 INT;
DECLARE id_hall_1 INT;
DECLARE id_hall_2 INT;
DECLARE id_hall_3 INT;
DECLARE id_hall_4 INT;
DECLARE id_hall_5 INT;
DECLARE id_movie_0 INT;
DECLARE id_movie_1 INT;
DECLARE id_movie_2 INT;
DECLARE id_movie_3 INT;
DECLARE id_movie_4 INT;
DECLARE id_movie_5 INT;
DECLARE id_movie_6 INT;
DECLARE id_movie_7 INT;
DECLARE id_movie_8 INT;
DECLARE id_movie_9 INT;

BEGIN
	SELECT h.id INTO id_hall_0
	FROM "public.Halls" h
	JOIN "public.Cinemas" c ON h.cinema_id = c.id
	WHERE c.name = 'Альфа' AND h.name = 'Г-401';

	SELECT h.id INTO id_hall_1
	FROM "public.Halls" h
	JOIN "public.Cinemas" c ON h.cinema_id = c.id
	WHERE c.name = 'Альфа' AND h.name = 'Г-402';

	SELECT h.id INTO id_hall_2
	FROM "public.Halls" h
	JOIN "public.Cinemas" c ON h.cinema_id = c.id
	WHERE c.name = 'Альфа' AND h.name = 'Г-403';

	SELECT h.id INTO id_hall_3
	FROM "public.Halls" h
	JOIN "public.Cinemas" c ON h.cinema_id = c.id
	WHERE c.name = 'Альфа' AND h.name = 'Г-405';

	SELECT h.id INTO id_hall_4
	